In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from MD_utils import read_xyz_file, trajectory_to_xyz

# MD Properties

## Excercise 0: Loading and Parameters

### Loading Data

In [ ]:
Trajectory = read_xyz_file("trajectory.xyz")  # Position of the atoms in angstrom
Trajectory.shape

### Simulation parameters

In [ ]:
N_atoms = Trajectory.shape[1]
dt_sampling = 10
dt_reduced = 0.005
rho_reduced = 0.84
N_box = 6

### Physical units parameters

In [ ]:
sigma_angstrom = 3.4
sigma_m = sigma_angstrom * 1e-10
eps_joule = 120 * 1.380649e-23
mass_kg = 39.948 * 1.66054e-27
tau = sigma_m * (mass_kg / eps_joule) ** 0.5  # in seconds
tau_ps = tau * 1e12  # convert to picoseconds

dt_ps = dt_reduced * tau_ps * dt_sampling  # in picoseconds

### Removing equilibration

In [ ]:
N_frames = 200
Trajectory = Trajectory[-N_frames:]
Trajectory.shape

### Time array and L_box

In [ ]:
time = np.arange(N_frames) * dt_ps  # time array in ps
L_box = ((4/rho_reduced)**(1./3.))* sigma_angstrom * N_box # L_box in angstroms

## Excercise 1:

In [ ]:
def unwrap_trajectory(trajectory_wrapped):
    '''
    Unwrap the trajectory!
    '''
    
    unwrapped = np.zeros_like(trajectory_wrapped)
    unwrapped[0] = trajectory_wrapped[0]

    jumps_cumulated = np.zeros((N_atoms, 3))  # counts boundary crossings

    for t in range(N_frames - 1):
        
        delta =                             # delta in real units
        
        current_jumps = np.rint(delta / L_box)

        
        # Update jumps_cumulated


        # Reconstruct next unwrapped position
        unwrapped[t + 1] = 

    return unwrapped

In [ ]:
unwrap_traj = unwrap_trajectory(Trajectory)

The function below allows you to create an xyz file from out Python arrays to visualize it in VMD!. Does your unwrapped trajectory looks reasonable? With the incorrect signs, the trajectory would look unreasonable...

In [ ]:
trajectory_to_xyz(unwrap_traj, filename="unwrap_trajectory.xyz")

In [ ]:
def compute_msd(unwrapped):
    '''
    Compute the mean squared displacement!
    '''
    r0 = unwrapped[0]  # initial positions

    displacements = unwrapped - r0  # displacement from initial position
    
    return msd


In [ ]:
msd = compute_msd(unwrap_traj)

In [ ]:
plt.plot(time, msd, label='MSD (Argon)')
plt.xlabel("Time [ps]")
plt.ylabel("MSD [$\mathrm{Å}^2$]")
plt.title("Mean Square Displacement of Argon")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Did you find a linear behaviour? What is the diffusion coefficient? What is the value of the difussion coefficient in cm^2/s?

#### To discuss with your neighrest neighbor(s):

- Do you think the data is from a system in solid, liquid or gas state?
- Can you discard any of these states from the plot?

# Excercise 2

In [ ]:
N_bins = 512
rmax = np.sqrt(3*L_box**2)/2
L_bin=rmax/N_bins
g_counter=np.zeros(N_bins)

In [ ]:
def counting_distances_frame(i):
    '''
    Add the distances to g_counter corresponding to the i-th frame
    '''
    rx = Trajectory[i][:,0];
    ry = Trajectory[i][:,1];
    rz = Trajectory[i][:,2];
    
    for k in range(N_atoms-1):
        j=k+1                

        # Distances to atoms with superior index (not normalized)
        dx = (rx[k]-rx[j:N_atoms])
        dy = (ry[k]-ry[j:N_atoms])
        dz = (rz[k]-rz[j:N_atoms])

        # Apply minimum image convention to dx, dy and dz

        

        # dx, dy and dz already with the minimum image convention in real units.
        r2 = dx*dx + dy*dy + dz*dz
        r = np.sqrt(r2)
        
        for corrected_distance in r:
            g_counter[?] += 2    # Find the expression for ?

Plot the unnormalized g(r) for the time frame 0! And compare it with the plots of the people who have already plotted it. We will see later that after certain r, we have a numerical artifact, so we are going to the plot for a range where this artifact is not present!

In [ ]:
# Plot g_counter for a specific time frame
rs = L_bin*range(N_bins) # Array for the plot
g_counter=np.zeros(N_bins)
counting_distances_frame(0)      # After this line g_counter has the counted and sorted the distances of the very first frame
plt.plot(rs[:250],g_counter[:250])
plt.xlabel("r [Å]")
plt.ylabel("g(r)")
plt.grid(True)
plt.title("Unnormalized g(r)")
plt.tight_layout()
plt.show()

### To discuss: 
- Does it change qualitatively when you plot it for different time frames?
- Can you explain its shape?
- What would be the correct normalization?

After normalization, may be it becomes clearer!

If you are very fast and you are convinced that your plot is correct you can try to normalize it! In the second half of the exercise, we create the array g_normalization, such that g_counter/g_normalization gives us the normalized radial distribution function g(r).

Let's normalize:

In [ ]:
g_normalization = np.zeros(N_bins)
for i in range(N_bins):
    g_normalization[i] =               # Compute the normalization such that g(r) = g_counter/g_normalization 

Let's plot again for one time frame. 

In [ ]:
g_counter=np.zeros(N_bins)
counting_distances_frame(0)
g_r_single_frame = g_counter/g_normalization
plt.plot(rs[:250], g_r_single_frame[:250])
plt.xlabel("r [Å]")
plt.ylabel("g(r)")
plt.title("g(r) for a single frame with wrong normalization")
plt.grid(True)
plt.tight_layout()

The normalization was made to average over all time frames too, so only the shape is relevant in the previous plot, not the exact values! 

And now the proper radial distribution function g(r):

In [ ]:
g_counter=np.zeros(N_bins)
for i in range(N_frames):
    counting_distances_frame(i)
g_r = g_counter/g_normalization)
plt.plot( r,g_r )

#### To discuss with your neighrest neighbor(s):

- Do you think the data is from a system in solid, liquid or gaseous state?
- How would you expect this plot for a system in a solid, a liquid and a gaseous state?

### In case you were too quick!

It seems that we have an artifact when we plot g(r) from 0 to rmax. At some radious the function g(r) starts decaying and it shoudln't. 
 - Can you identify where is the problem?
 - How would you modify g_(r) such that we avoid the artifact?

# Exercise 3

### Load Velocities

In [ ]:
Velocity = read_xyz_file("velocities.xyz")

### Removing equilibration

In [ ]:
Velocity = Velocity[-N_frames:]

Compute 

In [ ]:
def compute_vacf(vels):
    '''
    Compute the velocity autocorrelation function
    '''
    vacf = np.zeros(N_frames)
    for lag in range(N_frames):
        dot_sum = 0.0
        count = 0
        for t in range(N_frames - lag):
            v0 = vels[t]
            vlag = vels[t + lag]
            dot_sum +=           # Complete the function!
            count += N_atoms
        vacf[lag] = dot_sum / count
    return vacf

In [ ]:
vacf=compute_vacf(Velocity)

In [ ]:
plt.plot(time[:20], vacf[:20])
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MaxNLocator(10))  # up to 10 x-ticks
ax.yaxis.set_major_locator(ticker.MaxNLocator(10))  # up to 10 y-ticks
plt.grid(True)
plt.xlabel("VACF(τ)")
plt.ylabel("τ [ps]")
plt.grid(True)
plt.title("Velocity Autocorrelation Function")
plt.tight_layout()
plt.show()

### To discuss with your nearest neighbors:
    - What is the meaning of negative values of the VACF?
    - What shape of the velocity autocorrelation function would you expect for a system in solid, liquid and gaseous state?
    - What would be a system whose vacf(τ) = 1?
    - What would be a system whose vacf(τ) is a square wave function alternating between 1 and -1 with constant period and instantaneous transitions?

#### If you are too fast...
Instead of two np.sum(), write the missing line with one np.mean() and one np.sum(). You will need to modify the normalizations due to averages!